In [11]:

# Import Libraries and Define Paths
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from huggingface_hub import snapshot_download
from tqdm import tqdm
import os

# --- Define File Paths Based on Your Project Structure ---
PROCESSED_DATA_PATH = '../data/processed/filtered_complaints.csv'
VECTOR_STORE_PATH = '../vector_store/db_faiss'

# Ensure the vector store directory exists
os.makedirs(os.path.dirname(VECTOR_STORE_PATH), exist_ok=True)



In [7]:

# Load the Processed Data
try:
    df = pd.read_csv(PROCESSED_DATA_PATH)
    print("Processed data loaded successfully.")
    print(f"Dataset shape: {df.shape}")
    # Handle potential empty narratives that might have slipped through
    df.dropna(subset=['cleaned_narrative'], inplace=True)
    print(f"Shape after dropping any remaining NaNs: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: The file was not found at {PROCESSED_DATA_PATH}.")
    print("Please ensure you have run Task 1 successfully.")



Processed data loaded successfully.
Dataset shape: (363409, 8)
Shape after dropping any remaining NaNs: (363409, 8)


In [8]:

# Cell 3: Text Chunking

print("\n--- Starting Text Chunking ---")

# We use LangChain's RecursiveCharacterTextSplitter.
# This splitter tries to split text on a hierarchy of characters (like "\n\n", "\n", " ", "")
# to keep semantically related pieces of text together as much as possible.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The maximum size of each chunk (in characters)
    chunk_overlap=100   # The number of characters to overlap between chunks
)

# We will create a new list to hold our chunked documents.
# Each entry will be a LangChain 'Document' object, which holds the text
# and its associated metadata.

docs = []
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Chunking Narratives"):
    # Split the narrative into chunks
    chunks = text_splitter.split_text(row['cleaned_narrative'])
    
    for chunk in chunks:
        # For each chunk, we create a Document object and add metadata.
        # The metadata is crucial as it allows us to trace a retrieved chunk
        # back to its original complaint.
        docs.append(Document(
            page_content=chunk,
            metadata={
                'complaint_id': row['Complaint ID'],
                'product': row['Product'],
                'issue': row['Issue'],
                'company': row['Company'],
                'date_received': row['Date received']
            }
        ))

print(f"\nTotal number of documents created after chunking: {len(docs)}")
print("Example of a chunked document:")
print(docs[0])





--- Starting Text Chunking ---


Chunking Narratives: 100%|██████████| 363409/363409 [02:38<00:00, 2288.67it/s]


Total number of documents created after chunking: 588437
Example of a chunked document:
page_content='i made the mistake of using my wellsfargo debit card to depsit funds into atm machine outside their branch i went into the branch and was told they couldnt help and had to phone the customer service for help i did this and was told i was helped gave all the info for the time terminal id aact s was able to find the transaction and give me this info he said the dispute would take a few days i waited a few days and got a letter stating my dispute was rejected i went back into and they said they never got the transaction' metadata={'complaint_id': 14061897, 'product': 'Savings Account', 'issue': 'Managing an account', 'company': 'WELLS FARGO & COMPANY', 'date_received': '2025-06-13'}


In [12]:
# Cell 4: Embedding Model Setup

print("\n--- Setting up Embedding Model ---")

# We will use a pre-trained model from Hugging Face via the sentence-transformers library.
# 'all-MiniLM-L6-v2' is a fantastic starting model because it's:
# - Fast and lightweight.
# - Provides high-quality embeddings for semantic search.
# - Works well for general-purpose text.

# LangChain provides a convenient wrapper 'HuggingFaceEmbeddings'
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

print(f"Embedding model '{model_name}' loaded successfully.")




--- Setting up Embedding Model ---


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Embedding model 'sentence-transformers/all-MiniLM-L6-v2' loaded successfully.


In [ ]:

# Create and Persist the Vector Store
print("\n--- Creating and Saving the Vector Store ---")
# This is the most intensive step. It will:
# 1. Take each document chunk ('docs').
# 2. Use the 'embeddings' model to create a vector for each chunk.
# 3. Store the vector and its associated metadata in a FAISS index.

# The `FAISS.from_documents` method handles this entire process for us.
# NOTE: This can take several minutes and consume a significant amount of RAM.
vector_store = FAISS.from_documents(docs, embeddings)

# Save the created vector store locally.
vector_store.save_local(VECTOR_STORE_PATH)

print(f"\n✅ Vector store created and saved successfully at: {VECTOR_STORE_PATH}")





--- Creating and Saving the Vector Store ---


In [ ]:
# Cell 6: (Optional) Test the Vector Store

print("\n--- Testing the Vector Store ---")

# To verify it's working, let's load it back and perform a quick similarity search.
db = FAISS.load_local(VECTOR_STORE_PATH, embeddings, allow_dangerous_deserialization=True) # allow_dangerous_deserialization is needed for FAISS with Langchain

query = "My credit card was charged for something I did not buy"
results = db.similarity_search(query, k=3) # k is the number of results to return

print(f"Query: '{query}'")
print("\nTop 3 similar documents found:")
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(f"Product: {doc.metadata.get('product', 'N/A')}")
    print(f"Issue: {doc.metadata.get('issue', 'N/A')}")
    print(f"Source Complaint ID: {doc.metadata.get('complaint_id', 'N/A')}")
    print(f"Text Snippet: {doc.page_content[:400]}...")
    print("--------------------")